In [1]:
import tensorflow as tf

# Training samples path, change to your local path
training_samples_file_path = tf.keras.utils.get_file("trainingSamples.csv",
                                                     "https://raw.githubusercontent.com/dream-365/SparrowRecSys/master/src/main/"
                                                     "resources/webroot/sampledata/trainingSamples.csv")
# Test samples path, change to your local path
test_samples_file_path = tf.keras.utils.get_file("testSamples.csv",
                                                 "https://raw.githubusercontent.com/dream-365/SparrowRecSys/master/src/main/"
                                                 "resources/webroot/sampledata/testSamples.csv")

2024-03-31 07:22:10.874928: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-31 07:22:11.166929: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 07:22:11.166964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 07:22:11.206908: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-31 07:22:11.247143: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-31 07:22:11.248005: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# load sample as tf dataset
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12,
        label_name='label',
        na_value="0",
        num_epochs=1,
        ignore_errors=True)
    return dataset

In [ ]:
# split as test dataset and training dataset
train_dataset = get_dataset(training_samples_file_path)
test_dataset = get_dataset(test_samples_file_path)

# genre features vocabulary
genre_vocab = ['Film-Noir', 'Action', 'Adventure', 'Horror', 'Romance', 'War', 'Comedy', 'Western', 'Documentary',
               'Sci-Fi', 'Drama', 'Thriller',
               'Crime', 'Fantasy', 'Animation', 'IMAX', 'Mystery', 'Children', 'Musical']

GENRE_FEATURES = {
    'userGenre1': genre_vocab,
    'userGenre2': genre_vocab,
    'userGenre3': genre_vocab,
    'userGenre4': genre_vocab,
    'userGenre5': genre_vocab,
    'movieGenre1': genre_vocab,
    'movieGenre2': genre_vocab,
    'movieGenre3': genre_vocab
}

# all categorical features
categorical_columns = []
for feature, vocab in GENRE_FEATURES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    emb_col = tf.feature_column.embedding_column(cat_col, 10)
    categorical_columns.append(emb_col)
# movie id embedding feature
movie_col = tf.feature_column.categorical_column_with_identity(key='movieId', num_buckets=1001)
movie_emb_col = tf.feature_column.embedding_column(movie_col, 10)
categorical_columns.append(movie_emb_col)

# user id embedding feature
user_col = tf.feature_column.categorical_column_with_identity(key='userId', num_buckets=30001)
user_emb_col = tf.feature_column.embedding_column(user_col, 10)
categorical_columns.append(user_emb_col)

# all numerical features
numerical_columns = [tf.feature_column.numeric_column('releaseYear'),
                     tf.feature_column.numeric_column('movieRatingCount'),
                     tf.feature_column.numeric_column('movieAvgRating'),
                     tf.feature_column.numeric_column('movieRatingStddev'),
                     tf.feature_column.numeric_column('userRatingCount'),
                     tf.feature_column.numeric_column('userAvgRating'),
                     tf.feature_column.numeric_column('userRatingStddev')]

# cross feature between current movie and user historical movie
rated_movie = tf.feature_column.categorical_column_with_identity(key='userRatedMovie1', num_buckets=1001)
crossed_feature = tf.feature_column.indicator_column(tf.feature_column.crossed_column([movie_col, rated_movie], 10000))

# define input for keras model
inputs = {
    'movieAvgRating': tf.keras.layers.Input(name='movieAvgRating', shape=(), dtype='float32'),
    'movieRatingStddev': tf.keras.layers.Input(name='movieRatingStddev', shape=(), dtype='float32'),
    'movieRatingCount': tf.keras.layers.Input(name='movieRatingCount', shape=(), dtype='int32'),
    'userAvgRating': tf.keras.layers.Input(name='userAvgRating', shape=(), dtype='float32'),
    'userRatingStddev': tf.keras.layers.Input(name='userRatingStddev', shape=(), dtype='float32'),
    'userRatingCount': tf.keras.layers.Input(name='userRatingCount', shape=(), dtype='int32'),
    'releaseYear': tf.keras.layers.Input(name='releaseYear', shape=(), dtype='int32'),

    'movieId': tf.keras.layers.Input(name='movieId', shape=(), dtype='int32'),
    'userId': tf.keras.layers.Input(name='userId', shape=(), dtype='int32'),
    'userRatedMovie1': tf.keras.layers.Input(name='userRatedMovie1', shape=(), dtype='int32'),

    'userGenre1': tf.keras.layers.Input(name='userGenre1', shape=(), dtype='string'),
    'userGenre2': tf.keras.layers.Input(name='userGenre2', shape=(), dtype='string'),
    'userGenre3': tf.keras.layers.Input(name='userGenre3', shape=(), dtype='string'),
    'userGenre4': tf.keras.layers.Input(name='userGenre4', shape=(), dtype='string'),
    'userGenre5': tf.keras.layers.Input(name='userGenre5', shape=(), dtype='string'),
    'movieGenre1': tf.keras.layers.Input(name='movieGenre1', shape=(), dtype='string'),
    'movieGenre2': tf.keras.layers.Input(name='movieGenre2', shape=(), dtype='string'),
    'movieGenre3': tf.keras.layers.Input(name='movieGenre3', shape=(), dtype='string'),
}

# wide and deep model architecture
# deep part for all input features
deep = tf.keras.layers.DenseFeatures(numerical_columns + categorical_columns)(inputs)
deep = tf.keras.layers.Dense(128, activation='relu')(deep)
deep = tf.keras.layers.Dense(128, activation='relu')(deep)
# wide part for cross feature
wide = tf.keras.layers.DenseFeatures(crossed_feature)(inputs)
both = tf.keras.layers.concatenate([deep, wide])
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(both)
model = tf.keras.Model(inputs, output_layer)

# compile the model, set loss function, optimizer and evaluation metrics
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', tf.keras.metrics.AUC(curve='ROC'), tf.keras.metrics.AUC(curve='PR')])

# train the model
model.fit(train_dataset, epochs=5)

# evaluate the model
test_loss, test_accuracy, test_roc_auc, test_pr_auc = model.evaluate(test_dataset)
print('\n\nTest Loss {}, Test Accuracy {}, Test ROC AUC {}, Test PR AUC {}'.format(test_loss, test_accuracy,
                                                                                   test_roc_auc, test_pr_auc))

Instructions for updating:
Use `tf.data.Dataset.ignore_errors` instead.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` uti

/opt/conda/lib/python3.11/site-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['rating', 'timestamp', 'userRatedMovie2', 'userRatedMovie3', 'userRatedMovie4', 'userRatedMovie5', 'userAvgReleaseYear', 'userReleaseYearStddev'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


7403/7403 [==============================] - 58s 7ms/step - loss: 0.7253 - accuracy: 0.6103 - auc: 0.6308 - auc_1: 0.6668
Epoch 2/5
5091/7403 [===================>..........] - ETA: 16s - loss: 0.6063 - accuracy: 0.6775 - auc: 0.7288 - auc_1: 0.7521

In [5]:
# print some predict results
predictions = model.predict(test_dataset)
for prediction, goodRating in zip(predictions[:12], list(test_dataset)[0][1][:12]):
    print("Predicted good rating: {:.2%}".format(prediction[0]),
          " | Actual rating label: ",
          ("Good Rating" if bool(goodRating) else "Bad Rating"))

1870/1870 [==============================] - 5s 2ms/step
Predicted good rating: 21.18%  | Actual rating label:  Good Rating
Predicted good rating: 47.02%  | Actual rating label:  Good Rating
Predicted good rating: 91.56%  | Actual rating label:  Bad Rating
Predicted good rating: 28.52%  | Actual rating label:  Good Rating
Predicted good rating: 9.23%  | Actual rating label:  Good Rating
Predicted good rating: 42.87%  | Actual rating label:  Bad Rating
Predicted good rating: 22.97%  | Actual rating label:  Bad Rating
Predicted good rating: 79.92%  | Actual rating label:  Good Rating
Predicted good rating: 81.44%  | Actual rating label:  Bad Rating
Predicted good rating: 22.18%  | Actual rating label:  Bad Rating
Predicted good rating: 49.50%  | Actual rating label:  Bad Rating
Predicted good rating: 94.84%  | Actual rating label:  Good Rating


In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 movieAvgRating (InputLayer  [(None,)]                    0         []                            
 )                                                                                                
                                                                                                  
 movieGenre1 (InputLayer)    [(None,)]                    0         []                            
                                                                                                  
 movieGenre2 (InputLayer)    [(None,)]                    0         []                            
                                                                                                  
 movieGenre3 (InputLayer)    [(None,)]                    0         []                        

In [25]:

# Dense features layer
GENRE_EMB_LEN = 10
USER_EMB_LEN = 10
MOVIE_EMB_LEN = 10
USER_BUCKET_SIZE = 30001
MOVIE_BUCKET_SIZE = 1001
genre_emb_para_num = len(genre_vocab) * len(GENRE_FEATURES) * GENRE_EMB_LEN
user_id_emb_para_num = USER_BUCKET_SIZE * USER_EMB_LEN
movie_id_emb_para_num = MOVIE_BUCKET_SIZE * MOVIE_EMB_LEN

all_emb_para_num = genre_emb_para_num + user_id_emb_para_num + movie_id_emb_para_num

print(f'- genre_emb_para_num: {genre_emb_para_num}')
print(f'- user_id_emb_para_num: {user_id_emb_para_num}')
print(f'- movie_id_emb_para_num: {movie_id_emb_para_num}')
print(f'all_emb_para_num: {all_emb_para_num}')


output_col_num = len(numerical_columns) + len(GENRE_FEATURES) * GENRE_EMB_LEN + USER_EMB_LEN + MOVIE_EMB_LEN
print(f'output_col_num: {output_col_num}')

# Relu1
RELU1_OUTPUT_DIM = 128
relu_layer1_para_num = (output_col_num + 1) * RELU1_OUTPUT_DIM # +1是因为每个输入特征都与每个神经元连接，并且有一个额外的偏置项
print(f'relu_layer1_para_num: {relu_layer1_para_num}')

# Relu2
RELU2_OUTPUT_DIM = 128
relu_layer2_para_num = (RELU1_OUTPUT_DIM + 1) * RELU2_OUTPUT_DIM # +1是因为每个输入特征都与每个神经元连接，并且有一个额外的偏置项
print(f'relu_layer2_para_num: {relu_layer2_para_num}')

# Cross features (wide part)
cross_output_column_num = 10000

# concatenate wide and deep
concatenate_output_column_num = RELU2_OUTPUT_DIM + cross_output_column_num
print(f'concatenate_output_column_num: {concatenate_output_column_num}')

# sigmoid layer
sigmoid_layer_para_num = concatenate_output_column_num + 1 # +1是因为每个输入特征都与每个神经元连接，并且有一个额外的偏置项

print(f'sigmoid_layer_para_num: {sigmoid_layer_para_num}')

- genre_emb_para_num: 1520
- user_id_emb_para_num: 300010
- movie_id_emb_para_num: 10010
all_emb_para_num: 311540
output_col_num: 107
relu_layer1_para_num: 13824
relu_layer2_para_num: 16512
concatenate_output_column_num: 10128
sigmoid_layer_para_num: 10129
